In [6]:
import pandas as pd
import yfinance as yf
import numpy as np
from finvizfinance.screener.overview import Overview
from scipy.stats import linregress
import time

In [7]:
# ==========================================
# 1.(Finviz Screening)
# ==========================================
def get_filtered_picks(limit_per_country=5000): # Raised limit to capture more
    print("--- STEP 1: Fetching Universe from Finviz ---")
    
    try:
        foverview = Overview()
        
        # Criteria: USA, Strong Buy, High Liquidity, Not Penny Stocks
        filters_dict = {
            'Analyst Recom.': 'Strong Buy (1)',
            'Country': 'USA',
            'Average Volume': 'Over 2M', 
            'Market Cap.': '+Small (over $300mln)'
        }
        foverview.set_filter(filters_dict=filters_dict)
        df_results = foverview.screener_view()
        
        if df_results.empty:
            return pd.DataFrame()
            
        print(f"   Success! Found {len(df_results)} initial candidates.")
        
        # Clean up Column Names immediately
        if 'Analyst Recom' in df_results.columns:
            df_results.rename(columns={'Analyst Recom': 'Recom'}, inplace=True)
            
        # Ensure numeric Price for later
        df_results['Price'] = pd.to_numeric(df_results['Price'], errors='coerce')
        
        return df_results

    except Exception as e:
        print(f"   Error in Finviz Step: {e}")
        return pd.DataFrame()

In [8]:
# ==========================================
# 2. THE CREDIT MODEL (Z-Score & Margins)
# ==========================================
def calculate_z_score(info, financials, balance_sheet):
    """Calculates Altman Z-Score (Bankruptcy Risk)"""
    try:
        # Extract Key Metrics
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        
        # Z-Score Components
        working_capital = current_assets - current_liab
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        # Handle EBIT naming differences
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        market_cap = info.get('marketCap', 0)
        sales = financials.loc['Total Revenue'].iloc[0]

        # Ratios
        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        # Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E
        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    except:
        return np.nan

def get_margin_trend(financials):
    """Calculates 3-Year Gross Margin Slope"""
    try:
        years = financials.columns[:3]
        margins = []
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            if rev == 0 or np.isnan(rev): margins.append(0)
            else: margins.append(profit/rev)
            
        # Slope Calculation
        margins = margins[::-1] # Chronological order
        slope, _, _, _, _ = linregress(range(len(margins)), margins)
        
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
    except:
        return "N/A"


In [9]:
# ==========================================
# 3. EXECUTION ENGINE
# ==========================================
def run_screener():
    # A. Get the List
    candidates = get_filtered_picks()
    
    if candidates.empty:
        print("No stocks found to analyze.")
        return

    print(f"\n--- STEP 2: Running Credit Risk Model on {len(candidates)} Stocks ---")
    print("This may take 1-2 minutes (fetching balance sheets)...")
    
    scored_data = []
    
    # Loop through ALL candidates (No limit)
    for index, row in candidates.iterrows():
        ticker = row['Ticker']
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # Filter: Skip Financials (Banks use different Z-Score)
            sector = info.get('sector', 'Unknown')
            if 'Financial' in sector:
                bucket = "Financial (Skipped)"
                z_score = np.nan
                trend = "N/A"
            else:
                # Fetch Data
                fin = stock.financials
                bs = stock.balance_sheet
                
                if fin.empty or bs.empty:
                    continue
                
                # Run Model
                z_score = calculate_z_score(info, fin, bs)
                trend = get_margin_trend(fin)
                
                # Apply Buckets
                if (z_score > 2.99) and (trend in ["Improving", "Stable"]):
                    bucket = "Fortress (Safe)"
                elif (z_score < 1.8) and (trend == "Deteriorating"):
                    bucket = "Distress (AVOID)"
                elif (z_score < 1.8) and (trend == "Improving"):
                    bucket = "Moonshot (High Risk)"
                else:
                    bucket = "Middle of the Road"

            # Save Result
            scored_data.append({
                'Ticker': ticker,
                'Bucket': bucket,
                'Z-Score': z_score,
                'Margin_Trend': trend,
                'Sector': sector,
                'Finviz_Price': row['Price'],
                'Recom': row.get('Recom', 'N/A')
            })
            
            # Progress indicator (every 10 stocks)
            if index % 10 == 0:
                print(f"   Processed {index} / {len(candidates)}...", end='\r')

        except Exception:
            continue

    # B. Create Final Report
    final_df = pd.DataFrame(scored_data)
    
    # Sort: Best Fundamentals First
    final_df.sort_values(by='Z-Score', ascending=False, inplace=True)
    
    print(f"\n\n--- FINAL RISK REPORT ({len(final_df)} Stocks Analyzed) ---")
    
    # Display columns
    cols = ['Ticker', 'Bucket', 'Z-Score', 'Margin_Trend', 'Finviz_Price', 'Recom', 'Sector']
    display(final_df[cols])
    
    return final_df

# --- RUN THE WHOLE THING ---
risk_report = run_screener()

--- STEP 1: Fetching Universe from Finviz ---
   Success! Found 220 initial candidates.#######---] 10/11 

--- STEP 2: Running Credit Risk Model on 220 Stocks ---
This may take 1-2 minutes (fetching balance sheets)...


Exception ignored from cffi callback <function buffer_callback at 0x000001EEC4617A60>:
Traceback (most recent call last):
  File "C:\Users\jdcc3\AppData\Roaming\Python\Python313\site-packages\curl_cffi\curl.py", line 100, in buffer_callback
    @ffi.def_extern()
KeyboardInterrupt: 


: 

: 